The MIT License

Copyright 2025 Mohammad Parham Dehghan

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the “Software”), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED “AS IS”, WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.


In [1]:
!pip install flask==2.2.5 werkzeug==2.3.8 gtts==2.5.3 pyngrok==7.2.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.3/242.3 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 10.2 MB/s eta 0:00:00
  Attempting uninstall: werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3
  Attempting uninstall: click
    Found existing installation: click 8.2.1
    Uninstalling click-8.2.1:
      Successfully uninstalled click-8.2.1
  Attempting uninstall: flask
    Found existing installation: Flask 3.1.1
    Uninstalling Flask-3.1.1:
      Successfully uninstalled Flask-3.1.1


In [2]:
import os
import time
import threading
import json  # برای تبدیل دیکشنری به JSON
from flask import Flask, render_template_string, request
from gtts import gTTS
from pyngrok import ngrok
import glob
import logging

# تنظیم لاگ
logging.basicConfig(level=logging.INFO)

# تنظیم ngrok با توکن
ngrok.set_auth_token("2s9UYBDAhKCGblwUTV8PPf5AhJg_75PDRoniQ2hhzvtmoy8SN")

# برنامه Flask
app = Flask(__name__)

# دیکشنری ترجمه‌ها (همون قبلی)
translations = {
    'fa': {
        'title': 'تبدیل متن به گفتار',
        'placeholder': 'متن را وارد کنید...',
        'lang_options': {
            'fa': 'فارسی',
            'en': 'انگلیسی',
            'fr': 'فرانسوی',
            'de': 'آلمانی',
            'ar': 'عربی',
            'it': 'ایتالیایی',
            'ru': 'روسی',
            'es': 'اسپانیایی'
        },
        'speed_options': {
            'False': 'صدای معمولی',
            'True': 'صدای آهسته'
        },
        'button': '🔊 پخش صدا',
        'loading': 'در حال تولید فایل صوتی...',
        'ready': 'فایل صوتی آماده است',
        'error': 'خطا در تولید فایل صوتی: {}',
        'dir': 'rtl',
        'align': 'right'
    },
    'en': {
        'title': 'Text to Speech',
        'placeholder': 'Enter text...',
        'lang_options': {
            'fa': 'Persian',
            'en': 'English',
            'fr': 'French',
            'de': 'German',
            'ar': 'Arabic',
            'it': 'Italian',
            'ru': 'Russian',
            'es': 'Spanish'
        },
        'speed_options': {
            'False': 'Normal Speed',
            'True': 'Slow Speed'
        },
        'button': '🔊 Play Audio',
        'loading': 'Generating audio file...',
        'ready': 'Audio file is ready',
        'error': 'Error generating audio: {}',
        'dir': 'ltr',
        'align': 'left'
    },
    'fr': {
        'title': 'Texte à Parole',
        'placeholder': 'Entrez le texte...',
        'lang_options': {
            'fa': 'Persan',
            'en': 'Anglais',
            'fr': 'Français',
            'de': 'Allemand',
            'ar': 'Arabe',
            'it': 'Italien',
            'ru': 'Russe',
            'es': 'Espagnol'
        },
        'speed_options': {
            'False': 'Vitesse Normale',
            'True': 'Vitesse Lente'
        },
        'button': '🔊 Jouer Audio',
        'loading': 'Génération du fichier audio...',
        'ready': 'Fichier audio prêt',
        'error': 'Erreur lors de la génération audio: {}',
        'dir': 'ltr',
        'align': 'left'
    },
    'de': {
        'title': 'Text zu Sprache',
        'placeholder': 'Text eingeben...',
        'lang_options': {
            'fa': 'Persisch',
            'en': 'Englisch',
            'fr': 'Französisch',
            'de': 'Deutsch',
            'ar': 'Arabisch',
            'it': 'Italienisch',
            'ru': 'Russisch',
            'es': 'Spanisch'
        },
        'speed_options': {
            'False': 'Normale Geschwindigkeit',
            'True': 'Langsame Geschwindigkeit'
        },
        'button': '🔊 Audio Abspielen',
        'loading': 'Audiodatei wird generiert...',
        'ready': 'Audiodatei ist bereit',
        'error': 'Fehler beim Generieren des Audios: {}',
        'dir': 'ltr',
        'align': 'left'
    },
    'ar': {
        'title': 'تحويل النص إلى كلام',
        'placeholder': 'أدخل النص...',
        'lang_options': {
            'fa': 'فارسي',
            'en': 'إنجليزي',
            'fr': 'فرنسي',
            'de': 'ألماني',
            'ar': 'عربي',
            'it': 'إيطالي',
            'ru': 'روسي',
            'es': 'إسباني'
        },
        'speed_options': {
            'False': 'سرعة عادية',
            'True': 'سرعة بطيئة'
        },
        'button': '🔊 تشغيل الصوت',
        'loading': 'جاري إنشاء ملف الصوت...',
        'ready': 'ملف الصوت جاهز',
        'error': 'خطأ في إنشاء الصوت: {}',
        'dir': 'rtl',
        'align': 'right'
    },
    'it': {
        'title': 'Testo a Voce',
        'placeholder': 'Inserisci testo...',
        'lang_options': {
            'fa': 'Persiano',
            'en': 'Inglese',
            'fr': 'Francese',
            'de': 'Tedesco',
            'ar': 'Arabo',
            'it': 'Italiano',
            'ru': 'Russo',
            'es': 'Spagnolo'
        },
        'speed_options': {
            'False': 'Velocità Normale',
            'True': 'Velocità Lenta'
        },
        'button': '🔊 Riproduci Audio',
        'loading': 'Generazione file audio...',
        'ready': 'File audio pronto',
        'error': 'Errore nella generazione audio: {}',
        'dir': 'ltr',
        'align': 'left'
    },
    'ru': {
        'title': 'Текст в Речь',
        'placeholder': 'Введите текст...',
        'lang_options': {
            'fa': 'Персидский',
            'en': 'Английский',
            'fr': 'Французский',
            'de': 'Немецкий',
            'ar': 'Арабский',
            'it': 'Итальянский',
            'ru': 'Русский',
            'es': 'Испанский'
        },
        'speed_options': {
            'False': 'Нормальная скорость',
            'True': 'Медленная скорость'
        },
        'button': '🔊 Воспроизвести Аудио',
        'loading': 'Генерация аудиофайла...',
        'ready': 'Аудиофайл готов',
        'error': 'Ошибка генерации аудио: {}',
        'dir': 'ltr',
        'align': 'left'
    },
    'es': {
        'title': 'Texto a Voz',
        'placeholder': 'Ingrese texto...',
        'lang_options': {
            'fa': 'Persa',
            'en': 'Inglés',
            'fr': 'Francés',
            'de': 'Alemán',
            'ar': 'Árabe',
            'it': 'Italiano',
            'ru': 'Ruso',
            'es': 'Español'
        },
        'speed_options': {
            'False': 'Velocidad Normal',
            'True': 'Velocidad Lenta'
        },
        'button': '🔊 Reproducir Audio',
        'loading': 'Generando archivo de audio...',
        'ready': 'Archivo de audio listo',
        'error': 'Error al generar audio: {}',
        'dir': 'ltr',
        'align': 'left'
    }
}

# قالب HTML با JS برای تغییر دینامیک
template = """
<!doctype html>
<html lang="{{ lang }}">
  <head>
    <meta charset="utf-8">
    <title id="page-title">{{ trans['title'] }}</title>
    <style>
      body {
        font-family: sans-serif;
        text-align: {{ trans['align'] }};
        padding: 20px;
        background: #f4f7f9;
        direction: {{ trans['dir'] }};
        transition: all 0.3s ease;
      }
      h1 {
        color: #2c3e50;
        font-weight: bold;
        animation: fadeIn 0.5s;
      }
      .box {
        background: white;
        padding: 25px;
        border-radius: 15px;
        width: 90%;
        max-width: 600px;
        margin: auto;
        box-shadow: 0 4px 20px rgba(0,0,0,0.1);
        transition: box-shadow 0.3s;
      }
      .box:hover { box-shadow: 0 6px 25px rgba(0,0,0,0.15); }
      input, select, button {
        padding: 12px;
        margin: 12px 0;
        width: 100%;
        border-radius: 10px;
        border: 1px solid #bdc3c7;
        font-size: 16px;
        transition: border 0.3s;
      }
      input:focus, select:focus { border: 1px solid #3498db; }
      button {
        background: #27ae60;
        color: white;
        cursor: pointer;
        font-weight: bold;
        transition: background 0.3s, transform 0.2s;
      }
      button:hover { background: #219d54; transform: scale(1.02); }
      audio { margin-top: 20px; width: 100%; }
      .error { color: #e74c3c; font-weight: bold; }
      .loading { color: #3498db; animation: pulse 1s infinite; }
      @keyframes pulse { 0% { opacity: 1; } 50% { opacity: 0.5; } 100% { opacity: 1; } }
      @keyframes fadeIn { from { opacity: 0; } to { opacity: 1; } }
      /* Responsive for mobile */
      @media (max-width: 480px) {
        .box { padding: 15px; }
        h1 { font-size: 24px; }
      }
    </style>
    <script>
      const translations = {{ translations_json | safe }};

      function updateUI(lang) {
        const trans = translations[lang] || translations['fa'];

        // بروزرسانی عناصر
        document.getElementById('page-title').innerText = trans.title;
        document.querySelector('h1').innerText = '🎤 ' + trans.title;
        document.getElementById('text-input').placeholder = trans.placeholder;
        document.getElementById('submit-button').innerText = trans.button;

        // بروزرسانی گزینه‌های سرعت (slow)
        const slowSelect = document.getElementById('slow-select');
        slowSelect.options[0].text = trans.speed_options['False'];
        slowSelect.options[1].text = trans.speed_options['True'];

        // بروزرسانی گزینه‌های زبان (lang)
        const langSelect = document.getElementById('lang-select');
        for (let option of langSelect.options) {
          option.text = trans.lang_options[option.value];
        }

        // تغییر جهت و align
        document.body.style.direction = trans.dir;
        document.body.style.textAlign = trans.align;

        // انیمیشن fade-in برای عناصر
        document.querySelectorAll('h1, input, select, button').forEach(el => {
          el.style.animation = 'fadeIn 0.5s';
        });
      }

      function showLoading() {
        document.getElementById('status').innerText = translations[document.getElementById('lang-select').value].loading;
      }

      window.onload = function() {
        updateUI('{{ lang }}');  // اولیه
        document.getElementById('lang-select').addEventListener('change', function() {
          updateUI(this.value);
        });
      };
    </script>
  </head>
  <body>
    <div class="box">
      <h1>🎤 {{ trans['title'] }}</h1>
      <form method="POST" onsubmit="showLoading()">
        <input type="text" id="text-input" name="text" placeholder="{{ trans['placeholder'] }}" required><br>
        <select id="lang-select" name="lang">
          {% for code, name in trans['lang_options'].items() %}
            <option value="{{ code }}" {% if code == lang %}selected{% endif %}>{{ name }}</option>
          {% endfor %}
        </select><br>
        <select id="slow-select" name="slow">
          {% for val, name in trans['speed_options'].items() %}
            <option value="{{ val }}">{{ name }}</option>
          {% endfor %}
        </select><br>
        <button type="submit" id="submit-button">{{ trans['button'] }}</button>
      </form>
      <p id="status" class="{% if error %}error{% elif audio_file %}loading{% endif %}">
        {% if error %}{{ error }}{% elif audio_file %}{{ trans['ready'] }}{% endif %}
      </p>
      {% if audio_file %}
        <audio controls autoplay>
          <source src="{{ audio_file }}" type="audio/mp3">
        </audio>
      {% endif %}
    </div>
  </body>
</html>
"""

# تابع حذف فایل‌های قدیمی
def cleanup_old_files():
    while True:
        try:
            audio_files = glob.glob("static/audio/*.mp3")
            current_time = time.time()
            for file in audio_files:
                if current_time - os.path.getmtime(file) > 3600:
                    os.remove(file)
                    logging.info(f"Deleted old file: {file}")
        except Exception as e:
            logging.error(f"Error in cleanup: {e}")
        time.sleep(3600)

# روت اصلی
@app.route('/', methods=["GET", "POST"])
def index():
    lang = 'fa'  # پیش‌فرض
    audio_file = None
    error = None

    if request.method == "POST":
        text = request.form['text']
        lang = request.form['lang']
        slow = request.form['slow'] == 'True'

        try:
            filename = f"static/audio/{time.time()}.mp3"
            os.makedirs("static/audio", exist_ok=True)

            tts = gTTS(text=text, lang=lang, slow=slow)
            tts.save(filename)
            audio_file = "/" + filename
        except Exception as e:
            error = translations[lang]['error'].format(str(e))
            logging.error(error)

    trans = translations.get(lang, translations['fa'])
    translations_json = json.dumps(translations)  # برای JS

    return render_template_string(template, trans=trans, lang=lang, audio_file=audio_file, error=error, translations_json=translations_json)

# اجرای سرور
if __name__ == '__main__':
    try:
        http_tunnel = ngrok.connect(5000, bind_tls=True)
        public_url = http_tunnel.public_url
        print(f"Flask app running at: {public_url}")

        threading.Thread(target=cleanup_old_files, daemon=True).start()
        app.run(port=5000, debug=False)
    except Exception as e:
        logging.error(f"Error starting server: {e}")

Flask app running at: https://30148ccd0281.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [26/Jul/2025 17:10:00] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Jul/2025 17:10:01] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [26/Jul/2025 17:10:06] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Jul/2025 17:10:07] "GET /static/audio/1753549806.0540562.mp3 HTTP/1.1" 206 -
INFO:werkzeug:127.0.0.1 - - [26/Jul/2025 17:10:08] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [26/Jul/2025 17:10:30] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Jul/2025 17:10:31] "GET /static/audio/1753549830.642582.mp3 HTTP/1.1" 206 -
INFO:werkzeug:127.0.0.1 - - [26/Jul/2025 17:10:33] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [26/Jul/2025 17:11:24] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 -

In [ ]:
import os
os.kill(os.getpid(), 9)
